In [1]:
# Import standard library modules
import sys

# Set the relative path to the project root directory
relative_path_to_root = "../../../"

# Add the project root to the system path for importing in-house modules
sys.path.append(relative_path_to_root)

# Import in-house modules from the 'utilities' package
from utilities import load_data, save_data
from utilities import temporal_train_test_split
from utilities import print_title, print_label

In [2]:
# Data manipulation and analysis
import pandas as pd

# Date and time manipulation
from datetime import date

# File and directory manipulation
from pathlib import Path

# Data preprocessing and model selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

# Model evaluation metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import numpy as np

# Machine learning models (regression)
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor

# Ensemble methods
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingRegressor

In [3]:
def split_dataset_by_date(raw_data: pd.DataFrame, todays_date: str):
    # Filter data by today's date
    filter_data_by_date = raw_data["Date"] == todays_date
    
    # Create a new dataframe with today's data
    todays_data = raw_data[filter_data_by_date].reset_index(drop=True)
    
    # Create a new dataframe with historical data (excluding today's data)
    historical_data = raw_data[~filter_data_by_date].reset_index(drop=True)
    
    return historical_data, todays_data

In [4]:
def filter_data_by_date_range(data: pd.DataFrame, end_date: str):
    # Determine the start date as the minimum date in the dataframe
    start_date = data["Date"].min()
    
    # Create a date range tuple
    date_range = (start_date, end_date)
    
    # Create a filter to exclude data within the specified date range
    remove_data_by_date = (data["Date"] >= date_range[0]) & (data["Date"] < date_range[1])
    
    # Filter the dataframe using the created filter
    filtered_data = data[~remove_data_by_date].reset_index(drop=True)
    
    return filtered_data

In [5]:
file_name = "sp500_adj_close_raw_with_nas"
file_path = f"../../../data/raw_data/{file_name}"

raw_data = load_data(file_path)

╔═══════════════════════════════════════════════════════════════╗
║File `test_w_na.csv.bz2` loaded from `sp500_adj_close_raw_with_nas.zip`║
╚═══════════════════════════════════════════════════════════════╝


### Data Pre-Processing
___

#### Split todays data (For prediction) and historical data (For training)

In [6]:
todays_date = "2024-10-25"

historical_data, todays_data = split_dataset_by_date(raw_data, todays_date)

print("Todays Date:", todays_date)

Todays Date: 2024-10-25


#### Ensure Missing Values on Todays Data is what we are Predicting

In [7]:
todays_data.isnull().sum()

Date                    0
Ticker                  0
Adjusted Close          0
Next Day Close        501
Previous Day Close      0
Return                  0
Volatility              0
RSI                     0
SMA_50                  0
SMA_100                 0
SMA_200                 0
Upper Band              0
Lower Band              0
Support                 0
Resistance              0
Action                501
dtype: int64

#### Handle Missing Values (NA's)


In [8]:
historical_data.isnull().sum()

Date                       0
Ticker                     0
Adjusted Close             0
Next Day Close             0
Previous Day Close         0
Return                140634
Volatility            142254
RSI                   140913
SMA_50                     0
SMA_100                    0
SMA_200                    0
Upper Band                 0
Lower Band                 0
Support                    0
Resistance                 0
Action                141539
dtype: int64

In [9]:
# Remove all rows where `Adjusted Close` is 0
# Stock did not trade on these days because it did not exist yet
historical_data = historical_data[historical_data["Adjusted Close"] != 0].reset_index(drop=True)

historical_data.isnull().sum()

Date                     0
Ticker                   0
Adjusted Close           0
Next Day Close           0
Previous Day Close       0
Return                  81
Volatility            1701
RSI                    360
SMA_50                   0
SMA_100                  0
SMA_200                  0
Upper Band               0
Lower Band               0
Support                  0
Resistance               0
Action                1067
dtype: int64

In [10]:
# Remove all rows where `Volatility` is NaN
# This is because we don't have the necessary data to calculate volatility at these points
# We will not be able to use these rows for training

historical_data = historical_data.dropna(subset=["Volatility"]).reset_index(drop=True)

historical_data.isnull().sum()

Date                     0
Ticker                   0
Adjusted Close           0
Next Day Close           0
Previous Day Close       0
Return                   0
Volatility               0
RSI                    358
SMA_50                   0
SMA_100                  0
SMA_200                  0
Upper Band               0
Lower Band               0
Support                  0
Resistance               0
Action                1050
dtype: int64

In [11]:
# Backfill the `RSI` column
# This is because the RSI is calculated based on the previous day's data
historical_data["RSI"] = historical_data["RSI"].bfill()

historical_data.isnull().sum()

Date                     0
Ticker                   0
Adjusted Close           0
Next Day Close           0
Previous Day Close       0
Return                   0
Volatility               0
RSI                      0
SMA_50                   0
SMA_100                  0
SMA_200                  0
Upper Band               0
Lower Band               0
Support                  0
Resistance               0
Action                1050
dtype: int64

In [12]:
# Backfill the `Action` column (Trading Signals)
# This is because the trading signal is based on the previous day's data

historical_data["Action"] = historical_data["Action"].bfill()

historical_data.isnull().sum()

Date                  0
Ticker                0
Adjusted Close        0
Next Day Close        0
Previous Day Close    0
Return                0
Volatility            0
RSI                   0
SMA_50                0
SMA_100               0
SMA_200               0
Upper Band            0
Lower Band            0
Support               0
Resistance            0
Action                0
dtype: int64

In [13]:
# Print tickers that do not have any missing values
print_title("Tickers that do not have any missing values", closed_corners=False)

num_tickers = len(historical_data["Ticker"].unique())
print_label("Number of unique tickers:", num_tickers)

missing_val_filter = historical_data.isnull().any(axis=1)

# Group by ticker and check for missing values within each group
grouped = historical_data.groupby("Ticker")

# Tickers with no missing values
tickers_no_missing_values = grouped.filter(lambda x: not x.isnull().any().any())["Ticker"].unique()
print_label("Number of tickers with no missing values:", len(tickers_no_missing_values))

# Tickers with missing values
tickers_missing_values = grouped.filter(lambda x: x.isnull().any().any())["Ticker"].unique()
print_label("Number of tickers with missing values:", len(tickers_missing_values), closed_corners=True)

╔═══════════════════════════════════════════════════════════════╗
║          Tickers that do not have any missing values          ║
╠═══════════════════════════════════════════════════════════════╣
║ Number of unique tickers:      |          501.00000           ║
║ Number of tickers with no m... |          501.00000           ║
║ Number of tickers with miss... |           0.00000            ║
╚═══════════════════════════════════════════════════════════════╝


In [14]:
import numpy as np

# create numpy arraw with missing values and ticker
tickers_missing_val_count = np.array([(ticker, value) for ticker, value in historical_data[missing_val_filter]["Ticker"].value_counts().items()])

tickers_missing_val_count

array([], dtype=float64)

In [15]:
tickers_no_missing_values

array(['A', 'AAPL', 'ABT', 'ACGL', 'ACN', 'ADBE', 'ADI', 'ADM', 'ADP',
       'ADSK', 'AEE', 'AEP', 'AES', 'AFL', 'AIG', 'AIZ', 'AJG', 'AKAM',
       'ALB', 'ALGN', 'ALL', 'AMAT', 'AMD', 'AME', 'AMGN', 'AMP', 'AMT',
       'AMZN', 'ANSS', 'AON', 'AOS', 'APA', 'APD', 'APH', 'ARE', 'ATO',
       'AVB', 'AVY', 'AXON', 'AXP', 'AZO', 'BA', 'BAC', 'BALL', 'BAX',
       'BBY', 'BDX', 'BEN', 'BG', 'BIIB', 'BK', 'BKNG', 'BKR', 'BLDR',
       'BLK', 'BMY', 'BR', 'BRO', 'BSX', 'BWA', 'BX', 'BXP', 'C', 'CAG',
       'CAH', 'CAT', 'CB', 'CBRE', 'CCI', 'CCL', 'CDNS', 'CE', 'CF',
       'CHD', 'CHRW', 'CI', 'CINF', 'CL', 'CLX', 'CMCSA', 'CME', 'CMG',
       'CMI', 'CMS', 'CNC', 'CNP', 'COF', 'COO', 'COP', 'COR', 'COST',
       'CPB', 'CPRT', 'CPT', 'CRL', 'CRM', 'CSCO', 'CSGP', 'CSX', 'CTAS',
       'CTRA', 'CTSH', 'CVS', 'CVX', 'D', 'DAL', 'DD', 'DE', 'DECK',
       'DFS', 'DGX', 'DHI', 'DHR', 'DIS', 'DLR', 'DLTR', 'DOC', 'DOV',
       'DPZ', 'DRI', 'DTE', 'DUK', 'DVA', 'DVN', 'DXCM', 'EA', 'EBAY',


### Exploratory Data Analysis (EDA):
___

#### Todays Data:

In [16]:
print("Shape:", todays_data.shape)
display(todays_data.head(2))
display(todays_data.tail(2))

Shape: (501, 16)


,Date,Ticker,Adjusted Close,Next Day Close,Previous Day Close,Return,Volatility,RSI,SMA_50,SMA_100,SMA_200,Upper Band,Lower Band,Support,Resistance,Action
0,2024-10-25,A,130.16,NaN,130.69,-0.004055,0.012137,18.866226,140.33243,136.65535,137.94844,152.15059,129.75082,130.16,148.244,NaN
1,2024-10-25,AAPL,231.40,NaN,230.57,0.003600,0.013669,66.374374,226.80140,221.79668,200.76884,237.80908,222.31091,216.32,236.480,NaN


,Date,Ticker,Adjusted Close,Next Day Close,Previous Day Close,Return,Volatility,RSI,SMA_50,SMA_100,SMA_200,Upper Band,Lower Band,Support,Resistance,Action
499,2024-10-25,ZBRA,359.97,NaN,362.05,-0.005745,0.009788,43.769238,355.8908,336.8905,311.04135,380.01144,359.82355,320.77,377.68,NaN
500,2024-10-25,ZTS,179.91,NaN,181.50,-0.008760,0.012576,36.496883,189.0940,183.3149,179.24370,197.88783,182.27017,179.91,196.48,NaN


#### Historical Data:

In [17]:
print("Shape:", historical_data.shape)
display(historical_data.head(2))
display(historical_data.tail(2))

Shape: (1978479, 16)


,Date,Ticker,Adjusted Close,Next Day Close,Previous Day Close,Return,Volatility,RSI,SMA_50,SMA_100,SMA_200,Upper Band,Lower Band,Support,Resistance,Action
0,2008-01-02,A,23.256388,23.025743,23.538280,-0.011976,0.015704,48.82759,23.314175,23.299887,23.564934,24.727251,22.540236,21.392029,24.351946,short
1,2008-01-02,AAPL,5.876342,5.879055,5.974057,-0.016357,0.018937,59.06735,5.518483,4.939064,4.197630,6.135833,5.403559,4.637376,6.026840,buy


,Date,Ticker,Adjusted Close,Next Day Close,Previous Day Close,Return,Volatility,RSI,SMA_50,SMA_100,SMA_200,Upper Band,Lower Band,Support,Resistance,Action
1978477,2024-10-24,ZBRA,362.05,359.97,368.09,-0.016409,0.010532,44.39913,355.5556,336.31850,310.51785,379.32880,361.36920,320.77,377.68,short
1978478,2024-10-24,ZTS,181.50,179.91,188.99,-0.039632,0.013328,35.00878,189.1774,183.23051,179.33058,197.27248,184.37552,180.90,196.48,short


#### Optional: For initial phase of training, filter large dataset.

In [18]:
# remove_up_to = "2024-01-01"
# historical_data = filter_data_by_date_range(historical_data, remove_up_to)

# print("Shape:", historical_data.shape)
# display(historical_data.head())
# display(historical_data.tail())

#### Inspect Tickers Individually

In [19]:
# Inspect Cleaned data zeroing in on individual tickers
select_df = raw_data.copy()

select_ticker = select_df["Ticker"] == "AAPL"

select_df[select_ticker]

,Date,Ticker,Adjusted Close,Next Day Close,Previous Day Close,Return,Volatility,RSI,SMA_50,SMA_100,SMA_200,Upper Band,Lower Band,Support,Resistance,Action
1,2008-01-02,AAPL,5.876342,5.879055,5.974057,-0.016357,0.018937,59.067350,5.518483,4.939064,4.197630,6.135833,5.403559,4.637376,6.02684,buy
502,2008-01-03,AAPL,5.879055,5.430276,5.876342,0.000462,0.018287,56.259520,5.530891,4.960155,4.213231,6.122779,5.462215,4.637376,6.02684,sell
1003,2008-01-04,AAPL,5.430276,5.357593,5.879055,-0.076335,0.025363,37.311500,5.527205,4.975917,4.226226,6.142305,5.426252,4.637376,6.02684,short
1504,2008-01-07,AAPL,5.357593,5.164871,5.430276,-0.013385,0.024475,36.547832,5.522205,4.992085,4.238845,6.171195,5.360235,4.637376,6.02684,short
2005,2008-01-08,AAPL,5.164871,5.410674,5.357593,-0.035972,0.024816,36.241930,5.515250,5.007573,4.250567,6.214129,5.247783,4.637376,6.02684,buy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2118730,2024-10-21,AAPL,236.480000,235.860000,235.000000,0.006298,0.012880,66.655846,225.934400,220.221050,199.802290,236.515660,221.650340,216.320000,236.48000,sell
2119231,2024-10-22,AAPL,235.860000,230.760000,236.480000,-0.002622,0.012744,64.702080,226.301000,220.668960,200.079120,237.478500,221.536500,216.320000,236.48000,short
2119732,2024-10-23,AAPL,230.760000,230.570000,235.860000,-0.021623,0.013733,57.298534,226.490800,221.056290,200.308620,237.575040,221.878950,216.320000,236.48000,short
2120233,2024-10-24,AAPL,230.570000,231.400000,230.760000,-0.000823,0.013688,55.555565,226.667800,221.423930,200.539280,237.665250,222.093750,216.320000,236.48000,buy


### Create Multiple Versions of Dataset
___

In [20]:
main_data = historical_data.copy().reset_index(drop=True)

# Create multiple versions of the dataset

# Data with out dates and tickers (Set as index for reference)
data_v1 = main_data.copy().set_index(["Date", "Ticker"])

# Data with dates seperated into year, month, and day
data_v2 = main_data.copy()
data_v2["Date"] = pd.to_datetime(data_v2["Date"])
data_v2["Year"] = data_v2["Date"].dt.year
data_v2["Month"] = data_v2["Date"].dt.month
data_v2["Day"] = data_v2["Date"].dt.day
data_v2 = data_v2.set_index(["Date", "Ticker"])

In [21]:
data_v2

Adjusted Close  Next Day Close  Previous Day Close  \
Date       Ticker                                                       
2008-01-02 A            23.256388       23.025743           23.538280   
           AAPL          5.876342        5.879055            5.974057   
           ABT          18.130209       18.019754           18.240662   
           ACGL          7.608889        7.764444            7.816667   
           ACN          26.437077       25.982521           26.415081   
...                           ...             ...                 ...   
2024-10-24 XYL         130.650000      130.410000          131.720000   
           YUM         133.160000      133.040000          134.020000   
           ZBH         104.000000      102.330000          104.700000   
           ZBRA        362.050000      359.970000          368.090000   
           ZTS         181.500000      179.910000          188.990000   

                     Return  Volatility        RSI      SMA_50     SMA_100  \
Date       Ticker                                                            
2008-01-02 A      -0.011976    0.015704  48.827590   23.314175   23.299887   
           AAPL   -0.016357    0.018937  59.067350    5.518483    4.939064   
           ABT    -0.006055    0.010484  34.677372   18.138460   17.628250   
           ACGL   -0.026581    0.016022  45.154190    7.785511    7.878933   
           ACN     0.000833    0.024039  54.812176   26.577982   27.784420   
...                     ...         ...        ...         ...         ...   
2024-10-24 XYL    -0.008123    0.009364  41.637012  133.559020  134.482200   
           YUM    -0.006417    0.010962  32.368100  134.692780  133.522200   
           ZBH    -0.006686    0.010517  47.462685  107.512610  108.075780   
           ZBRA   -0.016409    0.010532  44.399130  355.555600  336.318500   
           ZTS    -0.039632    0.013328  35.008780  189.177400  183.230510   

                      SMA_200  Upper Band  Lower Band     Support  Resistance  \
Date       Ticker                                                               
2008-01-02 A        23.564934   24.727251   22.540236   21.392029   24.351946   
           AAPL      4.197630    6.135833    5.403559    4.637376    6.026840   
           ABT      17.709028   19.233107   18.221804   16.775562   19.134018   
           ACGL      7.874161    8.114465    7.378535    7.463333    8.307778   
           ACN      28.471031   28.227203   24.273775   24.765512   29.215677   
...                       ...         ...         ...         ...         ...   
2024-10-24 XYL     130.753770  138.216640  130.534350  126.710000  137.530000   
           YUM     133.979570  140.235460  130.624540  129.710000  139.920000   
           ZBH     115.475070  108.185260  101.411740  101.770000  115.912370   
           ZBRA    310.517850  379.328800  361.369200  320.770000  377.680000   
           ZTS     179.330580  197.272480  184.375520  180.900000  196.480000   

                  Action  Year  Month  Day  
Date       Ticker                           
2008-01-02 A       short  2008      1    2  
           AAPL      buy  2008      1    2  
           ABT     short  2008      1    2  
           ACGL      buy  2008      1    2  
           ACN      sell  2008      1    2  
...                  ...   ...    ...  ...  
2024-10-24 XYL     short  2024     10   24  
           YUM     short  2024     10   24  
           ZBH     short  2024     10   24  
           ZBRA    short  2024     10   24  
           ZTS     short  2024     10   24  

[1978479 rows x 17 columns]

#### Select which version of the data to work with

In [22]:
select_data = data_v2.copy()

select_columns_to_drop = ["Action", "Previous Day Close", "SMA_50", "Resistance", "Upper Band", "SMA_200"]

data = select_data.drop(columns=select_columns_to_drop)

print("Shape:", data.shape)
data.head()

Shape: (1978479, 11)


Adjusted Close  Next Day Close    Return  Volatility  \
Date       Ticker                                                         
2008-01-02 A            23.256388       23.025743 -0.011976    0.015704   
           AAPL          5.876342        5.879055 -0.016357    0.018937   
           ABT          18.130209       18.019754 -0.006055    0.010484   
           ACGL          7.608889        7.764444 -0.026581    0.016022   
           ACN          26.437077       25.982521  0.000833    0.024039   

                         RSI    SMA_100  Lower Band    Support  Year  Month  \
Date       Ticker                                                             
2008-01-02 A       48.827590  23.299887   22.540236  21.392029  2008      1   
           AAPL    59.067350   4.939064    5.403559   4.637376  2008      1   
           ABT     34.677372  17.628250   18.221804  16.775562  2008      1   
           ACGL    45.154190   7.878933    7.378535   7.463333  2008      1   
           ACN     54.812176  27.784420   24.273775  24.765512  2008      1   

                   Day  
Date       Ticker       
2008-01-02 A         2  
           AAPL      2  
           ABT       2  
           ACGL      2  
           ACN       2

### Split data features `X` and target `y`
___

In [23]:
# Split the data into features (X) and target (y)
X = data.drop(columns="Next Day Close")

y = data["Next Day Close"]

print("Shape of X:", X.shape)
print("Shape of y:", y.shape)

Shape of X: (1978479, 10)
Shape of y: (1978479,)


### Feature Engineering
___

#### Inspect Multicollinearity using VIF

In [24]:
# Perform correlation matrix of X
X.corr().style.background_gradient(cmap="coolwarm")

,Adjusted Close,Return,Volatility,RSI,SMA_100,Lower Band,Support,Year,Month,Day
Adjusted Close,1.000000,0.004575,-0.049952,0.023530,0.995365,0.998308,0.996711,0.259818,0.004643,0.000054
Return,0.004575,1.000000,0.026863,0.214930,-0.001790,-0.002142,-0.001604,0.001578,0.002862,-0.000203
Volatility,-0.049952,0.026863,1.000000,-0.095813,-0.039082,-0.059477,-0.057813,-0.137810,0.008778,-0.003205
RSI,0.023530,0.214930,-0.095813,1.000000,0.000401,0.007452,0.004387,0.011158,-0.002540,0.003985
SMA_100,0.995365,-0.001790,-0.039082,0.000401,1.000000,0.995424,0.997062,0.262989,0.005244,0.000356
Lower Band,0.998308,-0.002142,-0.059477,0.007452,0.995424,1.000000,0.997957,0.259315,0.004509,0.000082
Support,0.996711,-0.001604,-0.057813,0.004387,0.997062,0.997957,1.000000,0.260094,0.004889,0.000432
Year,0.259818,0.001578,-0.137810,0.011158,0.262989,0.259315,0.260094,1.000000,-0.028679,-0.001871
Month,0.004643,0.002862,0.008778,-0.002540,0.005244,0.004509,0.004889,-0.028679,1.000000,-0.001247
Day,0.000054,-0.000203,-0.003205,0.003985,0.000356,0.000082,0.000432,-0.001871,-0.001247,1.000000


##### **Note: It is recommended to remove `["Previous Day Close", "Resistance", "Upper Band", "SMA_50", "SMA_200"]` after VIF inspection...**

In [25]:
# Create a function to calculate VIF
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Set the threshold for VIF (Based on the domain knowledge of the data)
THRESHOLD = 2000

def highlight_vif(row):
    return ["background-color: black" if value < THRESHOLD else "" for value in row]

def calc_vif(X):
    # Calculate VIF values
    vif_values = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

    # Create a dataframe to display the VIF values
    vif = pd.DataFrame(
        data={"VIF": vif_values},
        index=X.columns
    )

    return vif

vif_df = calc_vif(X).sort_values("VIF")
vif_df.style.apply(highlight_vif)

,VIF
Return,1.055047
Volatility,3.424442
Day,4.226557
Month,4.641133
RSI,13.036789
Year,23.556780
SMA_100,236.750592
Adjusted Close,438.247347
Support,467.769801
Lower Band,620.094715


### Data Splitting
___

In [26]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2, # 80% training and 20% testing
    random_state=42
)

print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)

Shape of X_train: (1582783, 10)
Shape of X_test: (395696, 10)
Shape of y_train: (1582783,)
Shape of y_test: (395696,)


#### Inspect Probability Values `(p-values)`|

In [27]:
# Check P-Values
import statsmodels.api as sm

ols_model = sm.OLS(
    y_train,
    X_train
).fit()

ols_model

##### **Note: It is recommended to remove `["Day"]` after p-value inspection...**

In [28]:
def highlight_p_values(row):
    return ["background-color: black" if value <= 0.05 else "" for value in row]

p_values_df = ols_model.pvalues.sort_values().to_frame(name="p_value")

p_values_df.style.apply(highlight_p_values)

,p_value
Adjusted Close,0.000000
SMA_100,0.000000
Lower Band,0.000000
Support,0.000000
Year,0.000000
RSI,0.000000
Return,0.000000
Volatility,0.000003
Month,0.000131
Day,0.482955


### Model Training
___


#### Scale the data using `StandardScaler`

In [29]:
# Scale using StandardScaler
X_scaler = StandardScaler()
y_scaler = StandardScaler()

X_train_scaled = X_scaler.fit_transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

y_train_scaled = y_scaler.fit_transform(y_train.values.reshape(-1, 1))
y_test_scaled = y_scaler.transform(y_test.values.reshape(-1, 1))

print("X_train_scaled shape:", X_train_scaled.shape)
print("X_test_scaled shape:", X_test_scaled.shape)
print("y_train_scaled shape:", y_train_scaled.shape)
print("y_test_scaled shape:", y_test_scaled.shape)

X_train_scaled shape: (1582783, 10)
X_test_scaled shape: (395696, 10)
y_train_scaled shape: (1582783, 1)
y_test_scaled shape: (395696, 1)


In [30]:
def adj_r2_score(model, X, y):
    r2 = model.score(X, y)
    n_cols = X.shape[1]
    return 1 - (1 - r2) * (len(y) -1) / (len(y) - n_cols - 1)

In [31]:
lin_reg = LinearRegression(
    n_jobs=-1 # Use all processors
)

lin_reg.fit(X_train_scaled, y_train_scaled)


LinearRegression(n_jobs=-1)

In [32]:
def adj_r2_score(model, X, y):
    r2 = model.score(X, y)
    n_cols = X.shape[1]
    return 1 - (1 - r2) * (len(y) - 1) / (len(y) - n_cols - 1)

In [33]:
y_predict_scaled = lin_reg.predict(X_test_scaled)

y_predict_unscaled = y_scaler.inverse_transform(y_predict_scaled)
y_test_unscaled = y_scaler.inverse_transform(y_test_scaled)

mse = mean_squared_error(y_test_unscaled, y_predict_unscaled)
rmse = np.sqrt(mse)
r2 = r2_score(y_test_unscaled, y_predict_unscaled)
adj_r2 = adj_r2_score(lin_reg, X_test_scaled, y_test_scaled)

print_title("Linear Regression Model Evaluation", closed_corners=False)
print_label("Mean Squared Error:", mse)
print_label("Root Mean Squared Error:", rmse)
print_label("R-Squared:", r2)
print_label("Adjusted R-Squared:", adj_r2, closed_corners=True) 

╔═══════════════════════════════════════════════════════════════╗
║              Linear Regression Model Evaluation               ║
╠═══════════════════════════════════════════════════════════════╣
║ Mean Squared Error:            |           15.53391           ║
║ Root Mean Squared Error:       |           3.94131            ║
║ R-Squared:                     |           0.99965            ║
║ Adjusted R-Squared:            |           0.99965            ║
╚═══════════════════════════════════════════════════════════════╝


In [34]:
y_test_unscaled

array([[ 79.33535 ],
       [ 76.643776],
       [215.37453 ],
       ...,
       [ 65.32262 ],
       [ 32.881668],
       [ 84.435646]])

In [35]:
cv_scores = cross_val_score(
    LinearRegression(
        n_jobs=-1
    ),
    X_train_scaled,
    y_train_scaled,
    scoring="r2",
    cv=10
)

print_title("Cross Validation Scores", closed_corners=False)

for index, score in enumerate(cv_scores):
    print_label(f"Fold {index+1}:", score)

print_label("", "")
print_label("Mean R^2 Score:", cv_scores.mean())
print_label("Standard Deviation:", cv_scores.std(), closed_corners=True)

╔═══════════════════════════════════════════════════════════════╗
║                    Cross Validation Scores                    ║
╠═══════════════════════════════════════════════════════════════╣
║ Fold 1:                        |           0.99961            ║
║ Fold 2:                        |           0.99960            ║
║ Fold 3:                        |           0.99965            ║
║ Fold 4:                        |           0.99959            ║
║ Fold 5:                        |           0.99951            ║
║ Fold 6:                        |           0.99961            ║
║ Fold 7:                        |           0.99957            ║
║ Fold 8:                        |           0.99956            ║
║ Fold 9:                        |           0.99958            ║
║ Fold 10:                       |           0.99961            ║
║                                |                              ║
║ Mean R^2 Score:                |           0.99959            ║
║ Standard

### Predict tomorrows `Adjusted Close`
___

In [36]:
def preprocess_todays_data(df, columns_to_drop):
    # Convert 'Date' column to datetime
    df["Date"] = pd.to_datetime(df["Date"])
    
    # Extract year, month, and day from 'Date' column
    df["Year"] = df["Date"].dt.year
    df["Month"] = df["Date"].dt.month
    df["Day"] = df["Date"].dt.day
    
    # Drop specified columns
    df = df.drop(columns=columns_to_drop)
    
    # Set 'Date' and 'Ticker' as the index
    df = df.set_index(["Date", "Ticker"])
    
    # Drop 'Next Day Close' column
    df = df.drop(columns="Next Day Close")
    
    return df

X_to_predict = todays_data.copy()

X_to_predict = preprocess_todays_data(X_to_predict, select_columns_to_drop)

X_to_predict_scaled = X_scaler.transform(X_to_predict)

print("Shape of todays_data_clean_scaled:", X_to_predict_scaled.shape)
X_to_predict_scaled[:5]

Shape of todays_data_clean_scaled: (501, 10)


array([[ 0.21029221, -0.21050541, -0.50676969, -2.04849203,  0.26539134,
         0.25211298,  0.28301857,  1.62443221,  1.02782802,  1.06190002],
       [ 0.71656911,  0.1299215 , -0.38651463,  0.79365625,  0.70975971,
         0.74646253,  0.76314191,  1.62443221,  1.02782802,  1.06190002],
       [ 0.49863601, -0.4592691 , -0.79119435, -0.85020721,  0.50742677,
         0.5520643 ,  0.59719398,  1.62443221,  1.02782802,  1.06190002],
       [ 0.23249558,  0.58958269, -0.28334676,  0.37484342,  0.24850492,
         0.22527628,  0.19452787,  1.62443221,  1.02782802,  1.06190002],
       [ 0.13068012, -0.90781959, -0.64725082,  0.08741691,  0.12313012,
         0.15204863,  0.17207307,  1.62443221,  1.02782802,  1.06190002]])

In [37]:
y_to_predict_scaled = lin_reg.predict(X_to_predict_scaled)

y_to_predict_unscaled = y_scaler.inverse_transform(y_to_predict_scaled)

print("Shape of y_to_predict_unscaled:", y_to_predict_unscaled.shape)
y_to_predict_unscaled[:5]

Shape of y_to_predict_unscaled: (501, 1)


array([[130.28101605],
       [231.58869119],
       [187.95440641],
       [134.72715696],
       [114.33049032]])

In [48]:
prediction_df = todays_data.copy()

prediction_df["Next Day Close"] = y_to_predict_unscaled.round(3)

prediction_df = prediction_df[["Date", "Ticker", "Adjusted Close", "Next Day Close"]]

prediction_df = prediction_df.set_index(["Date", "Ticker"])

print("Shape:", prediction_df.shape)
display(prediction_df.head(20))
display(prediction_df.tail(20))

Shape: (501, 2)


Adjusted Close  Next Day Close
Date       Ticker                                
2024-10-25 A              130.160         130.281
           AAPL           231.400         231.589
           ABBV           187.820         187.954
           ABNB           134.600         134.727
           ABT            114.240         114.330
           ACGL           105.280         105.396
           ACN            360.790         361.005
           ADBE           483.730         484.730
           ADI            230.170         230.415
           ADM             56.570          56.591
           ADP            287.960         288.142
           ADSK           286.050         286.088
           AEE             87.800          87.843
           AEP             98.180          98.258
           AES             16.965          16.896
           AFL            109.710         109.787
           AIG             76.820          76.888
           AIZ            191.780         191.935
           AJG            283.280         283.617
           AKAM           101.980         102.057

Adjusted Close  Next Day Close
Date       Ticker                                
2024-10-25 WBD              7.590           7.549
           WDC             69.455          69.447
           WEC             97.170          97.225
           WELL           130.190         130.241
           WFC             64.520          64.537
           WM             206.800         207.080
           WMB             52.510          52.497
           WMT             82.510          82.560
           WRB             58.170          58.224
           WST            307.920         308.129
           WTW            290.410         290.662
           WY              31.785          31.783
           WYNN            97.980          98.019
           XEL             64.460          64.488
           XOM            119.510         119.567
           XYL            130.410         130.573
           YUM            133.040         133.129
           ZBH            102.330         102.496
           ZBRA           359.970         360.426
           ZTS            179.910         180.158

In [49]:
# Make index column a regular column
prediction_df.reset_index(inplace=True)

prediction_df

,Date,Ticker,Adjusted Close,Next Day Close
0,2024-10-25,A,130.16,130.281
1,2024-10-25,AAPL,231.40,231.589
2,2024-10-25,ABBV,187.82,187.954
3,2024-10-25,ABNB,134.60,134.727
4,2024-10-25,ABT,114.24,114.330
...,...,...,...,...
496,2024-10-25,XYL,130.41,130.573
497,2024-10-25,YUM,133.04,133.129
498,2024-10-25,ZBH,102.33,102.496
499,2024-10-25,ZBRA,359.97,360.426


In [50]:
file_name = "lin_reg_predict.zip"
file_path = f"../../../data/raw_data/{file_name}"

save_data(prediction_df, file_path)

╔═══════════════════════════════════════════════════════════════╗
║ File `lin_reg_predict.zip` already exists. Overwriting file.  ║
╚═══════════════════════════════════════════════════════════════╝
╔═══════════════════════════════════════════════════════════════╗
║        File saved and zipped as `lin_reg_predict.zip`         ║
╚═══════════════════════════════════════════════════════════════╝
